In [1]:
## import
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 


In [8]:
## reading in datasets 
df_mask = pd.read_csv('data/mask-use-by-county.csv')
df_covid = pd.read_csv('data/daily-cases-and-death.csv')

In [110]:
## changing df_mask county name 
df_mask = df_mask.rename(columns={'COUNTYFP': 'fips'})

In [71]:
df_covid[df_covid['county'] == 'San Diego']
df_covid = df_covid.sort_values(['state','county','date'])
df_covid = df_covid.reset_index(drop = True)
##reindex 

In [73]:
## add daily cases and daily deaths in df_covid 
df_covid['NewDailyCases']  = df_covid['cases'].diff()
df_covid['NewDailyDeaths'] = df_covid['deaths'].diff()

In [90]:
## setting value of daily cases and death to 0 when county changes 
df_covid.loc[df_covid['county'].ne(df_covid['county'].shift().bfill()), 'NewDailyCases'] = 0
df_covid.loc[df_covid['county'].ne(df_covid['county'].shift().bfill()), 'NewDailyDeaths'] = 0


In [94]:
## for some reason, there are negative values in daily cases and deaths still
df_covid[df_covid['NewDailyCases'] < 0].count()



date              13830
county            13830
state             13830
fips              12322
cases             13830
deaths            13830
NewDailyCases     13830
NewDailyDeaths    13830
dtype: int64

In [107]:
## inspection 
df_covid.loc[df_covid['NewDailyCases'] < 0, 'NewDailyCases'].value_counts()

-1.0      8969
-2.0      1798
-3.0       679
-4.0       343
-5.0       264
          ... 
-304.0       1
-264.0       1
-711.0       1
-423.0       1
-121.0       1
Name: NewDailyCases, Length: 279, dtype: int64

In [111]:
## regardless, merging these two datasets together. 
## leave cleaning NaN and negative values for later 

df_mask_covid = df_covid.merge(df_mask, how = 'outer', on ='fips' )

In [125]:
## there are a bunch of NaNs.... lets check them out! 
df_mask_covid[pd.isnull(df_mask_covid['fips'])].head()

,date,county,state,fips,cases,deaths,NewDailyCases,NewDailyDeaths,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
20953,2020-04-02,Unknown,Alaska,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
20954,2020-04-08,Unknown,Alaska,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
20955,2020-05-20,Unknown,Alaska,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
20956,2020-05-21,Unknown,Alaska,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
20957,2020-05-22,Unknown,Alaska,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


Seems like most of them are counties that the df_mask doesn't have the fips value of. Moving forward, decide to whether exclude those rows or add it from somewhere else